# 隐语义模型的梯度下降求解

### 1.算法实现

In [19]:
import numpy as np

"""
@输入参数：
R：M*N 的评分矩阵
P：初始化用户特征矩阵M*K
Q：初始化物品特征矩阵N*K
K：隐特征向量个数
steps: 最大迭代次数
alpha：步长
lamda：正则化系数

@输出：
分解之后的 P，Q
"""

def LFM_grad_desc(R, K=5, steps=3000, alpha=0.0002, lamda=0.004):
    M = len(R)
    N = len(R[0])
    P = np.random.rand(M,K)
    Q = np.random.rand(N,K)
    Q = Q.T
    
    for step in range(steps):
        for i in range(M):
            for j in range(N):
                # 如果评分大于0，表示有评分，才考虑误差
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - 2 * lamda * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - 2 * lamda * Q[k][j])

        # 根据更新之后的P、Q计算预测评分矩阵
        eR = np.dot(P,Q)
        # 计算当前损失函数
        e = 0
        for i in range(M):
            for j in range(N):
                if R[i][j] > 0:
                    e += (R[i][j]-np.dot(P[i,:],Q[:,j]))**2
                    for k in range(K):
                        e += lamda * (P[i][k]**2 + Q[k][j]**2)
        
        if e < 0.001:
            break
    return P, Q.T

### 2. 测试

In [20]:
R = np.array([[1,0,1,0,0],
              [0,1,0,0,1],
              [1,0,0,1,0],
              [0,0,1,0,0]])

nP,nQ = LFM_grad_desc(R)
R

array([[1, 0, 1, 0, 0],
       [0, 1, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0]])

In [21]:
nP

array([[ 0.1552958 ,  0.4671987 ,  0.57300613,  0.60611411,  0.33465085],
       [ 0.24276829,  0.49862254,  0.82293843,  0.13913211,  0.3202733 ],
       [-0.04162319,  0.49651099, -0.0013388 ,  0.60885689,  0.6311348 ],
       [ 0.09864814,  0.16555974,  0.85228676,  0.79004128,  0.52623308]])

In [22]:
nQ

array([[0.21608466, 0.10855826, 0.32945517, 0.81660581, 0.73945063],
       [0.11321961, 0.22313506, 0.93589454, 0.41266649, 0.11920088],
       [0.17955418, 0.61763027, 0.35939077, 0.29880653, 0.71711584],
       [0.84933477, 0.85311698, 0.37122398, 0.27773627, 0.68709738],
       [0.74862849, 0.90911455, 0.1804738 , 0.35901584, 0.48934658]])

In [23]:
nP.dot(nQ.T)

array([[1.01546924, 0.94811791, 0.94346746, 1.14146447, 1.02577339],
       [0.7281518 , 1.00452191, 0.91855755, 1.19577036, 0.99024236],
       [1.00835427, 0.43131013, 0.93323262, 0.99048565, 0.94741636],
       [1.3543552 , 1.23451262, 1.03971096, 1.12241289, 0.91932669]])